In [3]:
# from Ipython.display import display, Math, Latex
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
class LinReg(object) :
    '''
        Linear regression model
        -----------------------
        y = X@w
        X : feature matrix
        w : weight vector
        y : label vector
        
    '''
    
    def __init__(self) :
        self.t0 = 200
        self.t1 = 100000
        
    def predict(self, X:np.ndarray) -> np.ndarray :
        '''
            Prediction of output label for a given input.
            
            Args :
                X : Feature matrix.
            
            Returns :
                y : Output label predicted by the given input.
        '''
        y = X @ self.w
        return y
    
    def loss(self, X:np.ndarray) -> np.ndarray :
        '''
            Calculates the loss for a model based on known labels.
            
            Args :
                X : Feature matrix.
                y : Output label predicted by the given input.
                
            Returns :
                Loss
        '''
        
        e = y - self.predict(X)
        return (1/2) * (np.transpose(e) @ e)
        
    def rmse(self, X:np.ndarray, y:np.ndarray) :
        '''
            Calculates the root mean squared error of prediction w.r.t. actual label.
            
            Args :
                X : Feature matrix.
                y : Output label predicted by the given input.
                
            Returns :
                Loss
        '''

        return np.sqrt((2/X.shape[0] * self.loss(X, y)))
    
    def fit(self, X:np.ndarray, y:np.ndarray) -> np.ndarray :
        '''
            Estimates parameter of the linear regression model with normal equation.
            
            Args :
                X : Feature matrix.
                y : Output label predicted by the given input.
                
            Returns :
                Weight vector
        '''
        
#         linalg.pinv() is used to calculate the pseudo-inverse of a matrix which is equal to ((X)TX)^-1. 
        
        self.w = np.linalg.pinv(X) y
        return self.w
    
    
    def calculate_gradient(self, X:np.ndarray, y:np.ndarray) -> np.ndarray :
        '''
            Calculates gradients of loss function w.r.t weight vector on training set.
            
            Args :
                X : Feature matrix.
                y : Output label predicted by the given input.
                
            Returns :
                A vector of elements
        '''
        return np.transpose(X)@(self.predict(X) - y)
    
    def update_weights(self, grad:np.ndarray, lr:float) -> np.ndarray :
        '''
            Updating the weights based on the gradient of the loss function
            
            Weight updates are carried out with the following formula :
                
                w_new := w_old - lr*grad
            
            Args :
                grad : Gradient of loss w.r.t. w.
                lr : Learning rate.
                
            Returns :
                Updated weight vector
        '''
        return (self.w - lr*grad)
    
    
    
    
    
    
    
    
    
    